Ester Rivarola - 
Assignment 5 

##### 3/5/2018


In [1]:
#install all needed packages
#Codes used in class and in previous assignment as well

import time
from selenium import webdriver
from bs4 import BeautifulSoup
import collections
import nltk
import random
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.classify.util import accuracy
from nltk.classify import NaiveBayesClassifier
from nltk.classify import DecisionTreeClassifier

characters_to_remove = ["''",'``',"rt","https","’","“","”","\u200b","--","n't","'s","...","//t.c"]
wordnet_lemmatizer = WordNetLemmatizer()

def bag_of_words(words):
    words = word_tokenize(words)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in set(characters_to_remove)]
    words = [word for word in words if word not in stopwords.words("english")]
    words = [word for word in words if word not in set(string.punctuation[1:])]

    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in words]
    my_dict = dict([(word, True) for word in lemma_list])
    return my_dict

# Hotel 1: 


#### Doubletree By Hilton 

341 West 36th Street, New York City, NY 10018-640
    
Rating: 3.5

In [2]:
#code borrowed from last assignment 

browser = webdriver.Firefox()

dataset = []

for i in range(0,300,5):
    url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d1149404-Reviews-or' + str(i)
    browser.get(url)
    time.sleep(5)
    
    element_list = browser.find_elements_by_xpath("//span[@class='taLnk ulBlueLinks']")
        
    for e in element_list:
        try:
            e.click()
        except:
            pass
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    containers = soup.find_all('div','reviewSelector')
        
    for container in containers:
        int_rating = int(container.find('span','ui_bubble_rating')['class'][1].split('_')[1])//10
        if int_rating>3:
            rating = 'pos'
        else:
            rating = 'neg'
        
        review = container.p.text
        
        dataset.append((rating,review))

browser.quit()


In [3]:
#to check for distribution of reviews:

print('Positive: ' + str(sum(1 for t in dataset if t[0] == 'pos')))
print('Negative: ' + str(sum(1 for t in dataset if t[0] == 'neg')))


Positive: 119
Negative: 181


In [5]:
#to find the len of the dataset
len(dataset)

300

In [7]:
dataset_1 =[(bag_of_words(dataset[i][1]), dataset[i][0]) for i in range(len(dataset))]

random.shuffle(dataset_1)

train_set, test_set = dataset_1[:int(len(dataset_1)*.75)], dataset_1[int(len(dataset_1)*.75):]

In [8]:
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

In [9]:
from nltk.metrics.scores import (accuracy, precision, recall, f_measure)

### Naive Bayes Perfomance Metrics

In [10]:
tr_sets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    tr_sets[label].add(i)
    observed = nb_classifier.classify(feats)
    testsets[observed].add(i)
    

print('Accuracy:', float(round(nltk.classify.accuracy(nb_classifier, test_set),2)))
print('pos precision:', float(round(precision(tr_sets['pos'], testsets['pos']),2)))
print('pos recall:', float(round(recall(tr_sets['pos'], testsets['pos']),2)))
print('pos F-measure:', float(round(f_measure(tr_sets['pos'], testsets['pos']),2)))
print('neg precision:', float(round(precision(tr_sets['neg'], testsets['neg']),2)))
print('neg recall:', float(round(recall(tr_sets['neg'], testsets['neg']),2)))
print('neg F-measure:', float(round(f_measure(tr_sets['neg'], testsets['neg']),2)))

Accuracy: 0.75
pos precision: 0.7
pos recall: 0.8
pos F-measure: 0.75
neg precision: 0.8
neg recall: 0.7
neg F-measure: 0.75


### Decision Tress Perfomance Metrics

In [12]:
dt_classifier = DecisionTreeClassifier.train(train_set,binary=True,entropy_cutoff=.8,depth_cutoff=5,support_cutoff=30)

In [13]:
tr_sets1 = collections.defaultdict(set)
testsets1 = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    tr_sets1[label].add(i)
    observed = dt_classifier.classify(feats)
    testsets1[observed].add(i)
    
## Decision Tree Perfomance Metrics    
print('Accuracy:', float(round(nltk.classify.accuracy(dt_classifier, test_set),2)))
print('pos precision:', float(round(precision(tr_sets1['pos'], testsets1['pos']),2)))
print('pos recall:', float(round(recall(tr_sets1['pos'], testsets1['pos']),2)))
print('pos F-measure:', float(round(f_measure(tr_sets1['pos'], testsets1['pos']),2)))
print('neg precision:', float(round(precision(tr_sets1['neg'], testsets1['neg']),2)))
print('neg recall:', float(round(recall(tr_sets1['neg'], testsets1['neg']),2)))
print('neg F-measure:', float(round(f_measure(tr_sets1['neg'], testsets1['neg']),2)))


Accuracy: 0.69
pos precision: 0.88
pos recall: 0.4
pos F-measure: 0.55
neg precision: 0.64
neg recall: 0.95
neg F-measure: 0.77


For this assignment, the metric that would provide better insight is Negative Recall. Decision Trees provided the Highest Negegative Recall of 95%, this indicates that only 5% of the reviews were incorrecly classified as false negatives, and therefore I am choosing Decision Tress as my best performing model and now will try to transfer this knowledge to the second hotel. 

## HOTEL 2: 

#### Amsterdam Court Hotel

226 West 50th Street, New York City, NY 10019-6702

Rating: 3.5


In [14]:
browser = webdriver.Firefox()
dataset2 = []

for i in range(0,300,5):
    url2 = 'https://www.tripadvisor.com/Hotel_Review-g60763-d80075-Reviews-or' + str(i)
    browser.get(url2)
    time.sleep(5)
    
    element_list = browser.find_elements_by_xpath("//span[@class='taLnk ulBlueLinks']")
        
    for e in element_list:
        try:
            e.click()
        except:
            pass
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    containers = soup.find_all('div','reviewSelector')
        
    for container in containers:
        int_rating = int(container.find('span','ui_bubble_rating')['class'][1].split('_')[1])//10
        if int_rating>3:
            rating = 'pos'
        else:
            rating = 'neg'
        
        review = container.p.text
        
        dataset2.append((rating,review))

browser.quit()


In [15]:
#to check for distribution of reviews:

print('Positive: ' + str(sum(1 for t in dataset2 if t[0] == 'pos')))
print('Negative: ' + str(sum(1 for t in dataset2 if t[0] == 'neg')))

Positive: 185
Negative: 115


In [16]:
len(dataset2)

300

In [17]:
for_dataset2 =[(bag_of_words(dataset2[i][1]), dataset2[i][0]) for i in range(len(dataset2))]

random.shuffle(for_dataset2)

testset2 = for_dataset2

In [18]:
tr_sets3 = collections.defaultdict(set)
t_sets3 = collections.defaultdict(set)

for i, (feats, label) in enumerate(testset2):
    tr_sets3[label].add(i)
    observed1 = dt_classifier.classify(feats)
    t_sets3[observed1].add(i)
    

## Decision Tree Perfomance Metrics    
print('Accuracy:', float(round(nltk.classify.accuracy(dt_classifier, testset2),2)))
print('pos precision:', float(round(precision(tr_sets3['pos'], t_sets3['pos']),2)))
print('pos recall:', float(round(recall(tr_sets3['pos'], t_sets3['pos']),2)))
print('pos F-measure:', float(round(f_measure(tr_sets3['pos'], t_sets3['pos']),2)))
print('neg precision:', float(round(precision(tr_sets3['neg'], t_sets3['neg']),2)))
print('neg recall:', float(round(recall(tr_sets3['neg'], t_sets3['neg']),2)))
print('neg F-measure:', float(round(f_measure(tr_sets3['neg'], t_sets3['neg']),2)))



Accuracy: 0.54
pos precision: 0.76
pos recall: 0.38
pos F-measure: 0.51
neg precision: 0.45
neg recall: 0.81
neg F-measure: 0.58


The performace of each metrics had dropped but the negative recall is still signigicantly high 81% -- I would stop it here, but I want to compare this results with the the Naive Bayes Model too, to see if the metrics increases or decreases..let's see

In [19]:
tr_sets4 = collections.defaultdict(set)
t_sets4 = collections.defaultdict(set)

for i, (feats, label) in enumerate(testset2):
    tr_sets4[label].add(i)
    observed1 = nb_classifier.classify(feats)
    t_sets4[observed1].add(i)
    
    
## Naive Bayes Perfomance Metrics    
print('Accuracy:', float(round(nltk.classify.accuracy(nb_classifier, testset2),2)))
print('pos precision:', float(round(precision(tr_sets4['pos'], t_sets4['pos']),2)))
print('pos recall:', float(round(recall(tr_sets4['pos'], t_sets4['pos']),2)))
print('pos F-measure:', float(round(f_measure(tr_sets4['pos'], t_sets4['pos']),2)))
print('neg precision:', float(round(precision(tr_sets4['neg'], t_sets4['neg']),2)))
print('neg recall:', float(round(recall(tr_sets4['neg'], t_sets4['neg']),2)))
print('neg F-measure:', float(round(f_measure(tr_sets4['neg'], t_sets4['neg']),2)))

Accuracy: 0.73
pos precision: 0.72
pos recall: 0.91
pos F-measure: 0.8
neg precision: 0.75
neg recall: 0.43
neg F-measure: 0.55


Well..the level of accuracy increased, but the level of neg recall decreased dramatically. Therefore, I can conclude that the best model for this specific assignment is Decison Tree.